## Python für Fortgeschrittene 2

# Funktionales Programmieren I

### Programmierparadigmen

Programmierparadigma = grundlegende Sichtweise auf Programm + Daten → Programmierstil

* __Imperativ__ (dem Computer Befehle erteilen)

    * Befehle, (bedingte) Sprünge → Assembler
    * __strukturiert__ (+ Kontrollstrukturen wie Schleifen)
    * __prozedural__ (+ Unterprogramme / Prozeduren / Funktionen)  (Pascal, C)
    * __modular__ (+ Module als Gliederungseinheit)  (Modula)
    * __objektorientiert__ (Daten + Programm = Objekt; Kapselung, Vererbung, Polymorphie)

* __Deklarativ__ (dem Computer das Problem beschreiben)

    * __funktional__ (orientiert am mathematischen Funktionsbegriff)
    * __logisch__ (orientiert an der Prädikatenlogik)

* Weiterentwicklungen, Mischformen, orthogonale Konzepte

### Grundprinzipien funktionaler Programmierung

* Zentrales Strukturierungsmerkmal bilden __Funktionen__
* Funktion im mathematischen Sinne:

    * Funktion bildet Parameter auf (Funktions-, Rückgabe-) Wert ab
    * Rückgabewert nur von Parametern (+ ggf. Kontext) abhängig
    * keine Nebeneffekte (_reine_ funktionale Programmierung)
    
* __referentielle Transparenz__: Der Wert eines Ausdrucks hängt nur von seiner Umgebung ab, nicht vom Zeitpunkt der Auswertung.

    * unveränderliche Objekte _(immutable)_
    * keine (überschreibende) Zuweisungen (aber Definition/Variablenbindung)    

In [1]:
x = "langer oder komplexer Kram etc." # erlaubt, x wird an Wert gebunden
x = x + "bla"                         # verboten, x ändert Wert

* Funktionen als _first class citizen_

    * kann man herumreichen wie Daten
    * __Funktionen höherer Ordnung__, z.B. Sortieralgorithmus bekommt eine Vergleichsfunktion als Parameter, oder `map(f, S)` → Sequenz, in der `f` auf jedes Element aus `S` angewendet wurde
    
* Modell: Daten fließen durch eine Kette von Funktionen
* typisch: Arbeit auf _Sequenzen_ (die nicht zwingend materialisiert werden)
* typisch: unveränderliche, eher kurzlebige Objekte (statt langlebiger Objekte, die ihren Zustand ändern)
* __Rekursion__ zur Komplexitätskontrolle (und statt Iteration)

#### Vorteile des funktionalen Programmierens

* __Modularität__ (sehr kleine Funktionen, die leichter wiederzuverwenden und modular einzusetzen sind)
* Reduktion auf eine Abstraktionsebene, z.B. Trennung generischer Aufgaben (Iteration über Datenstruktur …) von spezifischer Funktion (z.B. berechne irgendwas)
* Parallelisierbarkeit (z.B. _Streams_ in Java 8; Map/Reduce)
* Formale Beweisbarkeit / Verifizierbarkeit (in Grenzen)
* leichtere Fehlersuche, leichtere Tests (kleine Funktionen, die nur von Parametern abhängig sind)

### Funktionales Programmieren in Python

Wie immer gilt in Python auch hier: Python ermöglicht die Verwendung des funktionalen Paradigmas, erzwingt es aber nicht durch Einschränkungen, wie es reine funktionale Programmiersprachen tun. Typischerweise verwendet man in Python prozedurale, objekt-orientierte und funktionale Verfahren, z.B. kann man objekt-orientiertes und funktionales Programmieren verwenden, indem man Funktionen definiert, die als Ein- und Ausgabe Objekte verwenden.

In Python wird das funktionale Programmieren u.a. durch folgende Komponenten realisiert:

* Funktionen, lambda-Ausdrücke
* Iterables & Iteratoren
* (List / Dictionary / Set / Generator) Comprehensions
* Generatoren
* map(), filter()
* itertools

### Iterables und Iteratoren

Python erwartet in bestimmten Kontexten ein iterierbares Objekt, z.B. in der for-Schleife:

In [4]:
a = range(3)
for i in a:
    print(str(i))

0
1
2


Welche Anforderungen muss a erfüllen, damit so eine `for`-Schleife funktionieren kann?

<p style="height:1024px;">...</p>

* `iter(iterable)` erzeugt einen Iterator                                         
* `next(iterator)` liefert den nächsten Wert oder Exception `StopIteration`                                         

In [24]:
iterable = range(3)
iterator = iter(iterable)

In [26]:
iterator

In [27]:
print(next(iterator))
print(next(iterator))
print(next(iterator))

0
1
2


In [28]:
print(next(iterator))

StopIteration: 

Das ist äquivalent zu 

In [29]:
for i in iter(a):
    print(str(i))

1
2
5
7


Man kann sich die vollständige Ausgabe eines Iterators ausgeben lassen, wenn man ihn als Parameter der list()- oder tuple() Funktion übergibt. 

In [5]:
#beispiel
a = [1, 2, 3,]
my_iterator = iter(a)
list(my_iterator)

[1, 2, 3]

In [10]:
my_iterator = iter(a)
tuple(my_iterator)

(0, 1, 2)

Frage: Warum habe ich im letzten Beispiel den Iterator neu erzeugt? Kann man das weglassen?

### Implementierung von Iterables & Iteratoren

* Iterable: Methode __iter__ liefert einen Iterator
* Iterator: Methode __next__ liefert das nächste Element

Dumme Implementierung eines Strings, über den man tokenweise iterieren kann:

In [6]:
class TokenString:
    
    def __init__(self, string):
        self.string = string
        self.tokens = self.string.split()
        
    def __iter__(self):
        return iter(self.tokens)
    

    
class _TokenIterator():
    # aus didaktischen Gründen ... 
    def __init__(self, tokenString):
        self.tokens = tokenString.tokens
        self.pos = -1
        
    def __next__(self):
        self.pos += 1
        if self.pos < len(self.tokens):
            return self.tokens[self.pos]
        else:
            raise StopIteration()

In [7]:
list(TokenString("Ich bin ein String aus Tokens"))

['Ich', 'bin', 'ein', 'String', 'aus', 'Tokens']

## Comprehensions

__Comprehensions__ erzeugen eine _iterable_ Python-Datenstruktur (Liste, Menge, Dictionary, Generator), indem sie deren Aufbau beschreiben. Die Schreibweise orientiert sich an der Mengenschreibweise in der Mathematik:

$$
SQ = \{ n² \mid n \in \mathbb{N} \}
$$

### List Comprehension

Eine _List Comprehension_ erzeugt eine Liste.

Beispiel: Quadratzahlen bis $9^2$. Zunächst mit einer klassischen `for`-Schleife:

In [16]:
squared = []
for x in range(10):
    squared.append(x**2)
squared

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Und hier die Version mit List Comprehension:

In [30]:
[x**2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Natürlich kann man den Rückgabewert von List Comprehensions auch in einer Variablen abspeichern.

In [31]:
squared = [x**2 for x in range(10)]
squared

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Vorteile:

* kürzere Schreibweise
* klarere Schreibweise (in so einem recht einfachen Beispiel)
* klar begrenzter Scope der Variablen, keine Nebeneffekte, keine Beeinflussung
* kein Reallozieren / Erweitern der Liste
* Optimierungspotential für Python

In [33]:
def squared_iter(iterable):
    squared = []
    for x in iterable:
        squared.append(x**2)
    return squared

def squared_comp(iterable):
    return [x**2 for x in iterable]

In [34]:
%timeit squared_iter(range(10000))

100 loops, best of 3: 3.04 ms per loop


In [35]:
%timeit squared_comp(range(10000))

100 loops, best of 3: 2.58 ms per loop


(`%timeit ausdruck` ist Jupyter-Syntactic Sugar für 'miss die Performanz von `ausdruck`, kapselt das Modul `timeit` aus der Python-Standardlibrary)

### Filter

Man kann das Iterable filtern. z.B. Quadratzahlen nur für gerade Zahlen:

In [41]:
[x**2 for x in range(10) if x % 2 == 0]

[0, 4, 16, 36, 64]

### Geschachtelte Schleifen

Man kann in list comprehensions auch mehrere geschachtelte for-Schleifen aufrufen:

In [36]:
#Aufgabe: vergleiche zwei Zahlenlisten und gebe alle Zahlenkombinationen aus, die ungleich sind
#Erst einmal die traditionelle Lösung mit geschachtelten Schleifen:
combs = [] 
for x in [1,2,3 ]: 
    for y in [3,1,4]: 
        if x != y: 
            combs.append((x, y))
combs

[(1, 3), (1, 4), (2, 3), (2, 1), (2, 4), (3, 1), (3, 4)]

Und nun als List Comprehension:

In [38]:
[(x,y) 
     for x in [1,2,3] 
     for y in [3,1,4]
     if x != y]

[(1, 3), (1, 4), (2, 3), (2, 1), (2, 4), (3, 1), (3, 4)]

<h4>Aufgabe 1</h4>
<p>Ersetzen Sie eine Reihe von Worten durch eine Reihe von Zahlen, die die Anzahl der Vokale anzeigen. Z.B.: "Dies ist ein Satz" -> "2 1 2 1". </p>


### Set- und Dictionary Comprehensions

Ein Äquivalent zu List Comprehensions gibt es auch für Mengen und Wörterbücher.

In [47]:
print("Liste:", [ x*y for x in range(10) for y in range(10)])
print("Menge:", { x*y for x in range(10) for y in range(10)})

Liste: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 0, 7, 14, 21, 28, 35, 42, 49, 56, 63, 0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 0, 9, 18, 27, 36, 45, 54, 63, 72, 81]
Menge: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15, 16, 18, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 54, 56, 63, 64, 72, 81}


In [54]:
{ n : 2**n for n in range(11) }

{0: 1, 1: 2, 2: 4, 3: 8, 4: 16, 5: 32, 6: 64, 7: 128, 8: 256, 9: 512, 10: 1024}

### Generator Comprehensions

Mit runden Klammern erzeugt man eine Generator Comprehension:

In [6]:
g = (x**2 for x in range(10))
g

<generator object <genexpr> at 0x7f9f9c0dd728>

Während List Comprehensions etc. immer eine vollständig materialisierte Liste erzeugen, ist dies bei Generator-Objekten nicht der Fall. Unser Generator-Objekt `g` ist in etwa äquivalent zu folgendem Objekt:

In [8]:
class SquareGenerator:
    def __init__(self):
        self.x_iterator = iter(range(10))
    def __iter__(self):
        return self
    def __next__(self):
        x = next(self.x_iterator)  
        return x**2

Mehr über Generatoren erfahren wir im nächsten Kapitel.

## Generatoren

Wir haben oben bereits _Generator Expressions_ kennengelernt, die ähnlich wie List Expressions funktionieren, aber keine vollständig materialisierte Liste erzeugen, sondern einen _Generator_:

In [3]:
r = range(100000000)
squares = [i**2 for i in r]  # Liste vollständig aufbauen -- dauert
print(squares[:10])

Mit einer _Generator Expression_ und der generischeren Funktion für Slicing, `islice` aus dem `itertools`-Package:

In [12]:
from itertools import islice
r = range(100000000)
squares = (i**2 for i in r)
print(list(islice(squares, 0, 10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


Squares ist ein Generator, und zugleich ein Iterator.

In [13]:
squares

<generator object <genexpr> at 0x7f59bc367a40>

Generatoren kann man nicht nur durch _Generator Expressions_ erzeugen, sondern auch mithilfe gewöhnlichem, imperativen Python-Code: __Eine Funktionsdefinition, die in Ihrem Body ein `yield`-Statement enthält, definiert eine Generatorfunktion. Eine Generatorfunktion erzeugt einen Generator.__ Hier ein einfaches Spielbeispiel:

In [31]:
def onetwothree():
    print("One: ")    # ← print-Statements aus didaktischen Gründen …
    yield 1
    print("Two: ")
    yield 2
    print("Three: ")
    yield 3
    print("Finished.")

Wenn wir die Funktion aufrufen, erhalten wir einen Generator:

In [25]:
g = onetwothree()
print(g)

<generator object onetwothree at 0x7f58dfc66e60>


Dieses Generatorobjekt können wir jetzt verwenden wie einen Iterator. Mit jedem Abruf des nächsten Elements wird die Funktion soweit ausgeführt, bis sie an ein `yield` kommt, und der Ausdruck rechts des Schlüsselworts ist dann das nächste Element:

In [26]:
next(g)

One: 


1

In [27]:
next(g)

Two: 


2

In [28]:
next(g)

Three: 


3

In [29]:
next(g)

Finished.


StopIteration: 

Generatoren können überall verwendet werden, wo ein Iterator/Iterable erwartet wird. Z.B. in Listen oder For-Schleifen.

In [30]:
print(list(onetwothree()))

One: 
Two: 
Three: 
Finished.
[1, 2, 3]


In [34]:
for i in onetwothree():
    print(i, '² = ', i**2, sep='')

One: 
1² = 1
Two: 
2² = 4
Three: 
3² = 9
Finished.


In [38]:
[i**2 for i in onetwothree()]

One: 
Two: 
Three: 
Finished.


[1, 4, 9]

Typischerweise verwendet man Generatoren mit Schleifen. Hier ein Beispiel (nach _Fluent Python_), das sowas wie `range` für beliebige Zahlentypen implementiert:

In [44]:
def progression(begin, step, end=None):
    common_type = type(begin + step)      # Ergebnistyp ermitteln (vgl. type(1+1.0))
    result = common_type(begin)           # für eine Sequenz mit konsistentem Typ
    forever = end == None                 # kein Ende angegeben? Endlos hochzählen ...
    index = 0
    
    while forever or result < end:
        yield result                      # aktuelles Ergebnis liefern
        index += 1
        result = begin + step*index       # Floating-Point-Fehler nicht aufsummieren ...
        
list(progression(0, 1/3, 2))

[0.0,
 0.3333333333333333,
 0.6666666666666666,
 1.0,
 1.3333333333333333,
 1.6666666666666665]

In [64]:
from fractions import Fraction
list(progression(0, Fraction(2,3), 3))

[Fraction(0, 1),
 Fraction(2, 3),
 Fraction(4, 3),
 Fraction(2, 1),
 Fraction(8, 3)]

#### Aufgabe G1

`enumerate()` zählt Objekte durch:

In [67]:
stuff = "Ich bin ein Test.".split()
print(list(enumerate(stuff)))

[(0, 'Ich'), (1, 'bin'), (2, 'ein'), (3, 'Test.')]


Bauen Sie `enumerate()` als Generator nach.

### `yield from`: Delegation in einem Iterator

Manchmal wollen Sie in einem Iterator selbst faktisch die Inhalte aus einem anderen Iterable einkopieren. Z.B. für unseren einfachen Tokenizer hier:

In [ ]:
import re
def tokenize(filename):
    WORD = re.compile(r"\w+")
    with open(filename, encoding="utf-8") as file:
        for line in file:
            for word in WORD.findall(line):
                yield word

Für die letzten beiden Zeilen gibt es eine einfachere Möglichkeit:

In [70]:
import re
def tokenize(filename):
    WORD = re.compile(r"\w+")
    with open(filename, encoding="utf-8") as file:
        for line in file:
            yield from WORD.findall(line)          

### Die Funktionen map(), filter(), und functools.reduce()

#### map()

map(FunktionX, Liste)<br/> 
Die Funktion FunktionX wird auf jedes Element der Liste angewandt. Ausgabe ist ein Iterator über eine neue Liste mit den Ergebnissen

In [61]:
a = ["ein Haus", "eine Tasse", "ein Kind"]
list(map(len, a))

[8, 10, 8]

In [62]:
[len(item) for item in a]

[8, 10, 8]

prozedurale Schreibweise:

In [63]:
lengths = []
for item in a:
    lengths.append(len(item))
lengths

[8, 10, 8]

#### Aufgabe 2

Verwenden Sie map() um in einer Liste von Worten jedes Wort in Großbuchstaben auszugeben. Diskutieren Sie evtl. Probleme mit einem Nachbarn. 

#### Aufgabe 3 (optional)

Lösen Sie Aufgabe 1 (Liste von Wörtern → Liste von Vokalanzahl pro Wort) mit map()

#### Aufgabe 3a

Analog, aber ermitteln Sie die Zahl der _unterschiedlichen_ Vokale pro Wort.

#### Aufgabe 3b

Analog, aber Ergebnis soll Liste der Wörter und ihrer Vokalzahl enthalten

#### filter()

filter(pred, Liste)<br/>
Die Funktion _pred_ wird auf jedes Element der Liste angewandt. Konstruiert einen neuen Iterator, in den die Elemente der Liste aufgenommen werden, für die die Funktion _pred_ ein wahres Ergebnis liefert.
<br/>Bsp.:

In [69]:
#returns True if x is an even number
def is_even(x): 
    return (x % 2) == 0 

b = [2,3,4,5,6]
list(filter(is_even, b))

[2, 4, 6]

#### Aufgabe 4

Verwenden Sie filter, um aus dem folgenden Text eine Wortliste zu erstellen, in der alle Pronomina, Artikel und die Worte "dass", "ist", "nicht", "auch", "und" nicht enthalten sind: <br/>
"Ich denke auch, dass ist nicht schlimm. Er hat es nicht gemerkt und das ist gut. Und überhaupt: es ist auch seine Schuld. Ehrlich, das ist wahr."

#### reduce()

`reduce(function, iterable [, initial])` aus dem `functools`-Modul funktioniert ähnlich `map`, reduziert die Liste jedoch auf einen einzelnen Wert. Die Verwendung ist umstritten.

In [17]:
from functools import reduce
def add(x, y):
    print("add({}, {}) -> {}".format(x, y, x+y)) # aus didaktischen Gründen ...
    return x+y

In [19]:
reduce(add, range(5))

add(0, 1) -> 1
add(1, 2) -> 3
add(3, 3) -> 6
add(6, 4) -> 10


10

### Das itertools-Modul

Das `itertools`-Modul liefert effiziente Implementierungen für Operationen auf Iteratoren und Iterables.


Die Funktionen des itertools-Moduls lassen sich einteilen in Funktionen, die: 
<ul>
<li>die einen neuen Iterator auf der Basis eines existierenden Iterators erzeugen. </li>
<li>die Teile der Ausgabe eines Iterators auswählen. </li>
<li>die die Ausgabe eines Iterators gruppieren.</li>
<li>die Iteratoren kombinieren</li>
</ul>

#### Neuen Iterator erzeugen

Diese Funktionen erzeugen einen neuen Iterator auf der Basis eines existierenden: <br/>
itertools.count(),itertools.cycle(), itertools.repeat(), itertools.chain(), itertools.isslice(), itertools.tee() 

itertools.cycle(iterator) Gibt die Liste der Elemente in iterator in einer unendlichen Schleife zurück

In [70]:
import itertools
#don't try this at home:
#list(itertools.cycle([1,2,3,4,5]))

itertools.repeat(iterator, [n]) wiederholt die Elemente in iterator n mal.

In [71]:
import itertools
list(itertools.repeat([1,2,3,4], 3))

[[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]]

itertools.chain(iterator_1, iterator_2, ...)  Erzeugt einen neuen Iterator, in dem die Elemente von iterator_1, _2 usw. aneinander gehängt sind.

In [72]:
a = [1, 2, 3]
b = [4, 5, 6]
c = [7, 8, 9]
list(itertools.chain(a, b, c))

[1, 2, 3, 4, 5, 6, 7, 8, 9]

#### Aufgabe 5

Verknüpfen Sie den Inhalt dreier Dateien zu einem Iterator

#### Teile der Ausgabe eines Iterators auswählen.

itertools.filterfalse(Prädikat, iterator) ist das Gegenstück zu filter(). Ausgabe enthält alle Elemente, für die das Prädikat falsch ist.

itertools.takewhile(Prädikat, iterator) - gibt solange Elemente aus, wie das Prädikat wahr ist

itertools.dropwhile(Prädikat, iter) entfernt alle Elemente, solange das Prädikat wahr ist. Gibt dann den Rest aus.

itertools.compress(Daten, Selektoren) Nimmt zwei Iteratoren und gibt nur die Elemente des ersten (Daten) zurück, für die das entsprechende Element im zweiten (Selektoren) wahr ist. Stoppt, wenn einer der Iteratoren erschöpft ist.

#### Iteratoren kombinieren

itertools.combinations(Iterator, r)  gibt alle r-Tuple Kombinationen der Elemente des Iterators wieder. Beispiel:

In [19]:
tuple(itertools.combinations([1, 2, 3, 4], 2))

((1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4))

itertools.permutations(iterator, r) gibt alle Permutationen aller Elemente unabhängig von der Reihenfolge in Iterator wieder: 

In [20]:
tuple(itertools.permutations([1, 2, 3, 4], 2)) 


((1, 2),
 (1, 3),
 (1, 4),
 (2, 1),
 (2, 3),
 (2, 4),
 (3, 1),
 (3, 2),
 (3, 4),
 (4, 1),
 (4, 2),
 (4, 3))

#### Aufgabe 7

Wieviele Zweier-Permutationen sind mit den graden Zahlen zwischen 1 und 101 möglich? 

#### The operator module

Funktionen für die gängigen Python-Operatoren etc.


Mathematische Operationen: add(), sub(), mul(), floordiv(), abs(),... <br/>
Logische Operationen: not_(), truth()<br/>
Bit Operationen: and_(), or_(), invert()<br/>
Vergleiche: eq(), ne(), lt(), le(), gt(), and ge()<br/> 
Objektidentität: is_(), is_not()<br/> 


In [21]:
a = [2, -3, 8, 12, -22, -1]
list(map(abs, a))

[2, 3, 8, 12, 22, 1]

### Lambda-Funktionen

lambda erlaubt es, kleine Funktionen anonym zu definieren. Nehmen wir an, wir wollen in einer List von Zahlen alle Zahlen durch 100 teilen und mit 13 multiplizieren. Dann könnten wir das so machen:

In [21]:
def calc(n):
    return (n * 13) / 100

a = [1, 2, 5, 7]
list(map(calc, a))

[0.13, 0.26, 0.65, 0.91]

Diese Funktion können wir mit Lambda nun direkt einsetzen:

In [22]:
list(map(lambda x: (x *  13)/100, a))

[0.13, 0.26, 0.65, 0.91]

`lambda`-Ausdrücke tragen ggf. nicht immer zur Lesbarkeit bei …

<ol>
<li>Write a lambda function.</li> 
<li>Write a comment explaining what the heck that lambda does. </li>
<li>Study the comment for a while, and think of a name that captures the essence of the comment. </li>
<li>Convert the lambda to a def statement, using that name. </li>
<li>Remove the comment. </li>
</ol>



### Hausaufgabe

1) Geben Sie alle Unicode-Zeichen zwischen 34 und 250 aus und geben Sie alle aus, die keine Buchstaben oder Zahlen sind

2) Wie könnte man alle Dateien mit der Endung *.txt in einem Unterverzeichnis hintereinander ausgeben? 

3) Schauen Sie sich in der Python-Dokumentation die Funktionen sort und itemgetter an. Wie kann man diese so kombinieren, dass man damit ein Dictionary nach dem value sortieren kann. (no stackoverflow :-)

4) Was ist der eleganteste/kürzeste/geekigste:-) Weg, den folgenden regelmäßig aufgebauten String zu erzeugen: `"0123 1234 2345 3456 4567 5678 6789"`

<br/><br/><br/><br/><br/><br/><br/><br/>

### Lösungen

#### Aufgabe 1

In [12]:
#zählt die Vokale eines strings
def cv(word):
    return len([ch for ch in word if ch in "aeiouAEIOUÄÖÜäöü"])

text = "Dies ist eine Lüge, oder nicht?"
[cv(word) for word in text.split()]

[2, 1, 3, 2, 2, 1]

<br/>
<br/><br/><br/><br/><br/>

#### Aufgabe 2

In [14]:
#uppeditys the string word 
def upper(word):
    return word.upper()

a = ["dies", "ist", "Ein", "satz"]
list(map(upper, a))

['DIES', 'IST', 'EIN', 'SATZ']

<br/><br/><br/><br/><br/><br/><br/><br/>

Aufgabe 3

In [16]:
def cv(word):
    return len([char for char in word if char.lower() in "aeiouäöü"])
    #return sum((1 for char in word if char in "aeiouAEIOUÄÖÜäöü"]))

a = "Dies ist eine Lüge, oder nicht?"

list(map(cv, a.split()))

[2, 1, 3, 2, 2, 1]

<br/><br/><br/><br/><br/><br/><br/><br/>

#### Aufgabe 4

In [11]:
import re

def is_no_function_word(word):
    """Returns True iff word is not a German function word."""
    f_words = ["der", "die", "das", "ich", "du", "er", "sie", "es", "wir", "ihr", "dass", "ist", "hat", "auch", "und", "nicht"]
    return word.lower() not in f_words
   
    
text = """Ich denke auch, dass ist nicht schlimm. Er hat es nicht gemerkt und das ist gut. 
          Und überhaupt: es ist auch seine Schuld. Ehrlich, das ist wahr."""

list(filter(is_no_function_word, re.findall("\w+", text)))

['denke',
 'schlimm',
 'gemerkt',
 'gut',
 'überhaupt',
 'seine',
 'Schuld',
 'Ehrlich',
 'wahr']